# Herramientas de inteligencia artificial
Roberto Alvarado

UTPL

Práctica 2

Hice una versión más completa del trabajo aqui 

# Consiguiendo datos del database y limpiadolos

In [7]:
import pandas as pd
#https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data
database = pd.read_csv("database/heart.csv")
database.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [8]:
#Preprocesando 
def get_categories( column ):
    labels = database[column].value_counts().index
    return dict(zip(labels,range(len(labels))))
get_categories("Sex")

{'M': 0, 'F': 1}

In [9]:
# Cmambiar todas las columndas a categorias numéricas
categorical_columns = set(database.columns)-set(database._get_numeric_data().columns)
clean_database = database.copy()
for col in categorical_columns:
    category = get_categories(col)
    print(col)
    print("New Category" + str(category))
    clean_database[col] = clean_database[col].replace(category)
clean_database.head()

ST_Slope
New Category{'Flat': 0, 'Up': 1, 'Down': 2}
ChestPainType
New Category{'ASY': 0, 'NAP': 1, 'ATA': 2, 'TA': 3}
ExerciseAngina
New Category{'N': 0, 'Y': 1}
Sex
New Category{'M': 0, 'F': 1}
RestingECG
New Category{'Normal': 0, 'LVH': 1, 'ST': 2}


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,2,140,289,0,0,172,0,0.0,1,0
1,49,1,1,160,180,0,0,156,0,1.0,0,1
2,37,0,2,130,283,0,2,98,0,0.0,1,0
3,48,1,0,138,214,0,0,108,1,1.5,0,1
4,54,0,1,150,195,0,0,122,0,0.0,1,0


# Dividiendo los testcases

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(
    clean_database.iloc[:,:-1], clean_database.iloc[:,-1], test_size=0.3, random_state=42
)

# Seteando el Flujo en MLFLOW y un random classifier 

In [11]:
!mkdir mlruns

mkdir: cannot create directory ‘mlruns’: File exists


In [12]:
import mlflow
import mlflow.sklearn
# Set up MLflow experiment (optional)
with mlflow.start_run():
    mlflow.set_experiment("randomforest_experiment")
    # Hyperparameters
    n_estimators = 100
    max_depth = 3
    random_state = 42

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    clf = RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth, random_state=random_state
    )
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, predictions)
    #Enviando los datos de precision a mlflow
    mlflow.log_metric("accuracy", acc) #type:ignore
    #Guardando info
    mlflow.sklearn.log_model(clf, "mlruns")
    print(f"Model logged with accuracy: {acc:.4f}")

MlflowException: Could not find experiment with ID 0

In [ ]:
# Testing
test = clean_database.sample(1)
cat = test["HeartDisease"].iloc[0]
feat = test.iloc[:,:-1]
print(cat)
print(clf.predict(feat))

# Para correr el servidor de MLFlow en el puerto 9090

In [ ]:
!mlflow server --default-artifact-root ./mlruns --host 0.0.0.0 --port 9090